In [1]:
import copy
import random

import numpy as np

In [2]:
WIDTH = 10
HEIGHT = 20
DISCOUNT = 0.9
LAMBDA = 0.8
FEATURE_COUNT = 2 * WIDTH + 2
PIECES = {
    'I': [
        np.array([
            [1, 1, 1, 1],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, 0, 0],
        ]),
        np.array([
            [1, 0, 0, 0],
            [1, 0, 0, 0],
            [1, 0, 0, 0],
            [1, 0, 0, 0],
        ])
    ],
    'T': [
        np.array([
            [1, 1, 1],
            [0, 1, 0],
            [0, 0, 0],
        ]),
        np.array([
            [1, 0, 0],
            [1, 1, 0],
            [1, 0, 0],
        ]),
        np.array([
            [0, 1, 0],
            [1, 1, 1],
            [0, 0, 0],
        ]),
        np.array([
            [0, 1, 0],
            [1, 1, 0],
            [0, 1, 0],
        ])
    ],
    'Z': [
        np.array([
            [1, 1, 0],
            [0, 1, 1],
            [0, 0, 0],
        ]),
        np.array([
            [0, 1, 0],
            [1, 1, 0],
            [1, 0, 0],
        ]),
    ],
    'S': [
        np.array([
            [0, 1, 1],
            [1, 1, 0],
            [0, 0, 0],
        ]),
        np.array([
            [1, 0, 0],
            [1, 1, 0],
            [0, 1, 0],
        ]),
    ],
    'O': [
        np.array([
            [1, 1],
            [1, 1],
        ])
    ],
    'L': [
        np.array([
            [1, 0, 0],
            [1, 0, 0],
            [1, 1, 0],
        ]),
        np.array([
            [0, 0, 1],
            [1, 1, 1],
            [0, 0, 0],
        ]),
        np.array([
            [1, 1, 0],
            [0, 1, 0],
            [0, 1, 0],
        ]),
        np.array([
            [1, 1, 1],
            [1, 0, 0],
            [0, 0, 0],
        ]),
    ],
    'J': [
        np.array([
            [0, 1, 0],
            [0, 1, 0],
            [1, 1, 0],
        ]),
        np.array([
            [1, 0, 0],
            [1, 1, 1],
            [0, 0, 0],
        ]),
        np.array([
            [1, 1, 0],
            [1, 0, 0],
            [1, 0, 0],
        ]),
        np.array([
            [1, 1, 1],
            [0, 0, 1],
            [0, 0, 0],
        ]),
    ],
}

PIECES_NAMES = PIECES.keys()

START_BOARD = [[0] * WIDTH for _ in range(HEIGHT)]

In [3]:
def can_be_placed_x_y(board, piece, x, y):
    n = len(piece)
    m = len(piece[0])
    mrange = range(m)
    for i in range(n):
        for j in mrange:
            if piece[i][j] == 1 and (x + i >= HEIGHT or y + j >= WIDTH or board[x + i][y + j] == 1):
                return False
    return True


def can_be_placed_y(board, piece, y):
    can_place_at = -1
    for i in range(HEIGHT):
        if can_be_placed_x_y(board, piece, i, y):
            can_place_at = i
        else:
            break
    return can_place_at


def place(board, piece, x, y):
    new_board = copy.deepcopy(board)
    n = len(piece)
    m = len(piece[0])
    mrange = range(m)
    for i in range(n):
        for j in mrange:
            if piece[i][j] == 1:
                new_board[x + i][y + j] = 1
    return new_board


def print_board(board, name=""):
    print(name)
    for row in board:
        print("".join("#" if x else "." for x in row))
    print("\n")


def get_piece_actions(board, piece):
    actions = []
    for rotation in piece:
        for i in range(WIDTH):
            placed = can_be_placed_y(board, rotation, i)
            if placed > -1:
                actions.append((placed, i, rotation))

    return actions


def get_reward(board):
    new_board = [row for row in board if not all(row)]
    cleared = HEIGHT - len(new_board)
    while len(new_board) < HEIGHT:
        new_board.insert(0, [0] * WIDTH)
    return new_board, cleared

In [4]:
def get_features(board):
    board = np.array(board)
    height, width = board.shape
    h = np.zeros(width, dtype=int)

    for k in range(width):
        column = board[:, k]
        non_zero = np.where(column == 1)[0]
        h[k] = height - non_zero[0] if len(non_zero) > 0 else 0

    height_diffs = np.abs(np.diff(h))

    max_h = np.max(h)

    holes = 0
    for k in range(width):
        col = board[:, k]
        filled_indices = np.where(col == 1)[0]
        if len(filled_indices) > 0:
            top = filled_indices[0]
            holes += np.sum(col[top + 1:] == 0)

    features = np.concatenate([
        [1.0],
        h,
        height_diffs,
        [max_h],
        [holes]
    ])

    return features

In [5]:
def get_value_with_weights(board, weights):
    features = get_features(board)
    return np.dot(features, weights)

In [6]:
def get_best_action_with_weights(weights, initial_board, initial_piece):
    actions = get_piece_actions(initial_board, PIECES[initial_piece])
    best_value = -1
    best_action = None
    best_reward = None
    best_new_board = None

    for action in actions:
        (x, y, rotation) = action
        new_board = place(initial_board, rotation, x, y)
        new_board, reward = get_reward(new_board)
        value = reward + get_value_with_weights(new_board, weights)
        if value > best_value:
            best_value = value
            best_action = action
            best_reward = reward
            best_new_board = new_board

    return best_action, best_value, best_reward, best_new_board

In [7]:
def get_trajectory(weights):
    trajectory = []

    new_board = START_BOARD

    counter = 0

    while counter < 1000:
        next_piece = random.choice(list(PIECES_NAMES))

        action, value, reward, updated_board = get_best_action_with_weights(weights, new_board, next_piece)

        if (action is not None):
            trajectory.append((new_board, action, reward, updated_board))
        else:
            trajectory.append((new_board, None, None, None))
            break

        new_board = updated_board
        counter += 1

    return trajectory

In [8]:
def get_current_value_estimates(weights, boards):
    value_estimates = []
    for board in boards:
        value_estimates.append(get_value_with_weights(board, weights))
    return value_estimates

In [9]:
def get_td_errors(value_estimates, rewards):
    errors = []
    for i in range(len(rewards)):
        error = rewards[i] + value_estimates[i + 1] - value_estimates[i]
        errors.append(error)
    return errors

In [10]:
def get_target(index, value_estimates, errors):
    total = 0

    total += value_estimates[index]

    for i in range(len(errors) - index):
        total += (LAMBDA ** i) * errors[index + i]

    return total

In [11]:
def get_targets(value_estimates, errors):
    targets = []

    for i in range(len(errors)):
        targets.append(get_target(i, value_estimates, errors))

    return targets

In [12]:
def get_fi_matrix(boards):
    fi_matrix = []

    for i in range(len(boards) - 1):
        fi_matrix.append(get_features(boards[i]))

    return fi_matrix

In [13]:
def get_fi_and_y(weights):
    trajectory = get_trajectory(weights)
    states = []
    rewards = []

    trajectory_len = len(trajectory)

    for index, (new_board, action, reward, updated_board) in enumerate(trajectory):
        states.append(new_board)
        if (index < trajectory_len - 1):
            rewards.append(reward)

    value_estimates = get_current_value_estimates(weights, states)
    errors = get_td_errors(value_estimates, rewards)
    targets = np.array(get_targets(value_estimates, errors))
    fi_matrix = np.array(get_fi_matrix(states))

    return (fi_matrix, targets, len(trajectory))


In [14]:
def get_updated_weights(weights, trajectory_count=100):
    fis = []
    ys = []

    for i in range(trajectory_count):
        fi, y, trajectory_len = get_fi_and_y(weights)
        fis.append(fi)
        ys.append(y)

    fi = np.vstack(fis)
    y = np.concatenate(ys)

    new_r, _, _, _ = np.linalg.lstsq(fi, np.array(np.array(y)), rcond=None)
    return new_r

In [15]:
def play_with_weights(weights):
    total_reward = 0

    new_board = START_BOARD

    while True:
        next_piece = random.choice(list(PIECES_NAMES))

        action, _, reward, updated_board = get_best_action_with_weights(weights, new_board, next_piece)

        if (action is None):
            break

        new_board = updated_board
        total_reward += reward

    return total_reward

In [16]:
def play_avg_reward(weights, count=50):
    total_reward = 0
    for _ in range(count):
        total_reward += play_with_weights(weights)
    return total_reward / count

In [ ]:
weights = np.zeros(FEATURE_COUNT)

for i in range(50):
    print("iteration", i, flush=True)
    print(play_avg_reward(weights), flush=True)
    new_weights = get_updated_weights(weights)
    weights = new_weights

print(play_avg_reward(weights), flush=True)
print(weights)